# 01. Data Preparation & Event Modeling

## Objective
Prepare clean, event-level behavioral data for churn analysis by combining
user activity, feature usage, support interactions, and billing events.

This notebook focuses only on data cleaning and transformation,
not analysis.


In [2]:
import pandas as pd
import numpy as np
from pathlib import Path


C:\Users\DELL\anaconda3\Lib\site-packages\pandas\core\computation\expressions.py:22: UserWarning: Pandas requires version '2.10.2' or newer of 'numexpr' (version '2.10.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [8]:
from pathlib import Path

# Project root (current working directory)
BASE_PATH = Path(".")

RAW_DATA_PATH = BASE_PATH / "01_Data" / "raw"
PROCESSED_DATA_PATH = BASE_PATH / "01_Data" / "processed"

PROCESSED_DATA_PATH.mkdir(parents=True, exist_ok=True)

RAW_DATA_PATH, PROCESSED_DATA_PATH


(WindowsPath('01_Data/raw'), WindowsPath('01_Data/processed'))

In [9]:
list(RAW_DATA_PATH.iterdir())



[WindowsPath('01_Data/raw/billing.csv'),
 WindowsPath('01_Data/raw/events.csv'),
 WindowsPath('01_Data/raw/users.csv')]

Load CSV Files

In [10]:
import pandas as pd

users_df = pd.read_csv(RAW_DATA_PATH / "users.csv")
events_df = pd.read_csv(RAW_DATA_PATH / "events.csv")
billing_df = pd.read_csv(RAW_DATA_PATH / "billing.csv")

users_df.shape, events_df.shape, billing_df.shape


((3, 3), (6, 3), (3, 4))

In [11]:
display(users_df.head())
display(events_df.head())
display(billing_df.head())


,user_id,signup_date,plan
0,U001,2024-01-10,Basic
1,U002,2024-01-15,Premium
2,U003,2024-02-01,Basic


,user_id,event_type,event_date
0,U001,login,2024-03-01
1,U001,feature_use,2024-03-02
2,U001,login,2024-03-10
3,U002,login,2024-03-01
4,U002,support_ticket,2024-03-05


,user_id,billing_date,amount,status
0,U001,2024-03-01,10,paid
1,U002,2024-03-01,20,paid
2,U003,2024-03-01,10,failed


Convert Date Columns

In [13]:
users_df['signup_date'] = pd.to_datetime(users_df['signup_date'])
events_df['event_date'] = pd.to_datetime(events_df['event_date'])
billing_df['billing_date'] = pd.to_datetime(billing_df['billing_date'])


Get Last Activity per User

In [14]:
last_event = (
    events_df
    .groupby('user_id')['event_date']
    .max()
    .reset_index()
    .rename(columns={'event_date': 'last_event_date'})
)

last_event


,user_id,last_event_date
0,U001,2024-03-10
1,U002,2024-03-05
2,U003,2024-02-15


Get Last Successful Payment

In [15]:
last_payment = (
    billing_df[billing_df['status'] == 'paid']
    .groupby('user_id')['billing_date']
    .max()
    .reset_index()
    .rename(columns={'billing_date': 'last_payment_date'})
)

last_payment


,user_id,last_payment_date
0,U001,2024-03-01
1,U002,2024-03-01


Build Churn Base Table

In [16]:
churn_base = (
    users_df
    .merge(last_event, on='user_id', how='left')
    .merge(last_payment, on='user_id', how='left')
)

churn_base


,user_id,signup_date,plan,last_event_date,last_payment_date
0,U001,2024-01-10,Basic,2024-03-10,2024-03-01
1,U002,2024-01-15,Premium,2024-03-05,2024-03-01
2,U003,2024-02-01,Basic,2024-02-15,NaT


Churn Flag & Inactivity Metrics

Set a Reference Date

In [17]:
import pandas as pd

REFERENCE_DATE = pd.to_datetime("2024-04-01")
REFERENCE_DATE


Timestamp('2024-04-01 00:00:00')

Calculate Inactivity Days

In [18]:
churn_base['days_since_last_event'] = (
    REFERENCE_DATE - churn_base['last_event_date']
).dt.days

churn_base['days_since_last_payment'] = (
    REFERENCE_DATE - churn_base['last_payment_date']
).dt.days

churn_base


,user_id,signup_date,plan,last_event_date,last_payment_date,days_since_last_event,days_since_last_payment
0,U001,2024-01-10,Basic,2024-03-10,2024-03-01,22,31.0
1,U002,2024-01-15,Premium,2024-03-05,2024-03-01,27,31.0
2,U003,2024-02-01,Basic,2024-02-15,NaT,46,NaN


Define Churn Flag (Business Rule)

In [19]:
churn_base['churn_flag'] = (
    (churn_base['days_since_last_event'] > 30) &
    (churn_base['days_since_last_payment'] > 30)
).astype(int)

churn_base


,user_id,signup_date,plan,last_event_date,last_payment_date,days_since_last_event,days_since_last_payment,churn_flag
0,U001,2024-01-10,Basic,2024-03-10,2024-03-01,22,31.0,0
1,U002,2024-01-15,Premium,2024-03-05,2024-03-01,27,31.0,0
2,U003,2024-02-01,Basic,2024-02-15,NaT,46,NaN,0


Move Reference Date Forward

In [21]:
REFERENCE_DATE = pd.to_datetime("2024-05-15")

churn_base['days_since_last_event'] = (
    REFERENCE_DATE - churn_base['last_event_date']
).dt.days

churn_base['days_since_last_payment'] = (
    REFERENCE_DATE - churn_base['last_payment_date']
).dt.days

churn_base['churn_flag'] = (
    (churn_base['days_since_last_event'] > 30) &
    (churn_base['days_since_last_payment'] > 30)
).astype(int)

churn_base


,user_id,signup_date,plan,last_event_date,last_payment_date,days_since_last_event,days_since_last_payment,churn_flag
0,U001,2024-01-10,Basic,2024-03-10,2024-03-01,66,75.0,1
1,U002,2024-01-15,Premium,2024-03-05,2024-03-01,71,75.0,1
2,U003,2024-02-01,Basic,2024-02-15,NaT,90,NaN,0


Quick Churn Summary

In [22]:
churn_base['churn_flag'].value_counts(normalize=True) * 100


churn_flag
1    66.666667
0    33.333333
Name: proportion, dtype: float64

Early Warning Signals (Pre-Churn)

Create Risk Signals

In [23]:
churn_base['low_engagement'] = churn_base['days_since_last_event'] > 14
churn_base['payment_risk'] = churn_base['days_since_last_payment'] > 14

churn_base[['user_id', 'low_engagement', 'payment_risk']]


,user_id,low_engagement,payment_risk
0,U001,True,True
1,U002,True,True
2,U003,True,False


Risk Scoring (No ML)

In [24]:
churn_base['risk_score'] = (
    churn_base['low_engagement'].astype(int) +
    churn_base['payment_risk'].astype(int)
)

churn_base[['user_id', 'risk_score', 'churn_flag']]


,user_id,risk_score,churn_flag
0,U001,2,1
1,U002,2,1
2,U003,1,0


Risk Distribution

In [25]:
churn_base['risk_score'].value_counts().sort_index()


risk_score
1    1
2    2
Name: count, dtype: int64

Silent Churn Detection

Silent Churn Flag

In [26]:
churn_base['silent_churn'] = (
    (churn_base['days_since_last_event'] > 21) &
    (churn_base['days_since_last_payment'] <= 30)
).astype(int)

churn_base[['user_id', 'silent_churn']]


,user_id,silent_churn
0,U001,0
1,U002,0
2,U003,0


Revenue at Risk

Monthly Revenue per User

In [29]:
import pandas as pd
from pathlib import Path

RAW_DATA_PATH = Path("01_Data/raw")

billing = pd.read_csv(RAW_DATA_PATH / "billing.csv")
billing


,user_id,billing_date,amount,status
0,U001,2024-03-01,10,paid
1,U002,2024-03-01,20,paid
2,U003,2024-03-01,10,failed


Monthly Revenue per User

In [30]:
monthly_revenue = (
    billing
    .groupby('user_id')['amount']
    .sum()
    .reset_index()
)

monthly_revenue.rename(
    columns={'amount': 'monthly_revenue'},
    inplace=True
)

monthly_revenue


,user_id,monthly_revenue
0,U001,10
1,U002,20
2,U003,10


Merge Revenue with Churn Base

In [31]:
churn_base = churn_base.merge(
    monthly_revenue,
    on='user_id',
    how='left'
)

churn_base


,user_id,signup_date,plan,last_event_date,last_payment_date,days_since_last_event,days_since_last_payment,churn_flag,low_engagement,payment_risk,risk_score,silent_churn,monthly_revenue
0,U001,2024-01-10,Basic,2024-03-10,2024-03-01,66,75.0,1,True,True,2,0,10
1,U002,2024-01-15,Premium,2024-03-05,2024-03-01,71,75.0,1,True,True,2,0,20
2,U003,2024-02-01,Basic,2024-02-15,NaT,90,NaN,0,True,False,1,0,10


Revenue at Risk

In [32]:
revenue_at_risk = churn_base.loc[
    churn_base['risk_score'] >= 1,
    'monthly_revenue'
].sum()

revenue_at_risk


np.int64(40)

In [33]:
churn_base[['user_id','risk_score','churn_flag','silent_churn','monthly_revenue']]


,user_id,risk_score,churn_flag,silent_churn,monthly_revenue
0,U001,2,1,0,10
1,U002,2,1,0,20
2,U003,1,0,0,10


Executive Summary Cell

## Executive Summary – Churn Intelligence

- X% of users are currently churned
- Y% are in early risk stage
- Silent churn users detected who are still paying but disengaged
- ₹Z monthly revenue is at risk due to churn

This analysis enables proactive retention without machine learning.



Retention Decision Engine

Define Risk Buckets

In [34]:
def risk_bucket(score):
    if score >= 2:
        return "High Risk"
    elif score == 1:
        return "Medium Risk"
    else:
        return "Low Risk"

churn_base['risk_bucket'] = churn_base['risk_score'].apply(risk_bucket)

churn_base[['user_id', 'risk_score', 'risk_bucket']]


,user_id,risk_score,risk_bucket
0,U001,2,High Risk
1,U002,2,High Risk
2,U003,1,Medium Risk


Define Intervention Strategy

In [35]:
def retention_action(row):
    if row['risk_bucket'] == "High Risk":
        return "Immediate support call + 20% discount"
    elif row['risk_bucket'] == "Medium Risk":
        return "Feature reminder + usage nudge"
    else:
        return "No action"

churn_base['recommended_action'] = churn_base.apply(
    retention_action, axis=1
)

churn_base[['user_id', 'risk_bucket', 'recommended_action']]


,user_id,risk_bucket,recommended_action
0,U001,High Risk,Immediate support call + 20% discount
1,U002,High Risk,Immediate support call + 20% discount
2,U003,Medium Risk,Feature reminder + usage nudge


Simulate Retention Impact

Assumed Save Rates

In [37]:
SAVE_RATE = {
    "High Risk": 0.40,     # 40% can be saved
    "Medium Risk": 0.20,   # 20% can be saved
    "Low Risk": 0.00
}

churn_base['save_probability'] = churn_base['risk_bucket'].map(SAVE_RATE)


Revenue Saved Simulation

In [38]:
churn_base['expected_revenue_saved'] = (
    churn_base['monthly_revenue'] * churn_base['save_probability']
)

churn_base[['user_id', 'risk_bucket', 'monthly_revenue', 'expected_revenue_saved']]


,user_id,risk_bucket,monthly_revenue,expected_revenue_saved
0,U001,High Risk,10,4.0
1,U002,High Risk,20,8.0
2,U003,Medium Risk,10,2.0


Total Business Impact

In [40]:
total_revenue_at_risk = churn_base.loc[
    churn_base['risk_score'] >= 1,
    'monthly_revenue'
].sum()

expected_revenue_saved = churn_base['expected_revenue_saved'].sum()

total_revenue_at_risk, expected_revenue_saved


(np.int64(40), np.float64(14.0))